# Imports

In [1]:
import pandas as pd
import numpy as np

---

# Carga de datos

In [2]:
auctions = pd.read_pickle("../../../data/tp2/auctions_tp2_formateado.pkl")

---

# Armado de validaciones

In [3]:
for validacion_nro in range(1,7):
    auctions["validacion_{}".format(validacion_nro)] = (((20+validacion_nro) <= auctions["date"].dt.day) & ((auctions["date"].dt.day) <= (22+validacion_nro)))

---

# Obtención del tiempo hasta aparición ("label") para cada ventana de datos

In [4]:
tres_dias = 3*24*60*60
for validacion_nro in range(1,7):
    validacion = auctions.loc[auctions["validacion_{}".format(validacion_nro)]].groupby("device_id").agg({"date" : "min"}).reset_index()
    validacion["target"] = validacion["date"] - np.datetime64('2019-04-{} 00:00:00.000000'.format(20+validacion_nro))
    del validacion["date"]
    validacion["target"] = validacion["target"].map(lambda x: (x.delta / 1000000000))
    target = auctions.loc[auctions["ventana_{}".format(validacion_nro)]]["device_id"].drop_duplicates().to_frame()
    target = target.reset_index()
    del target["index"]
    target = target.merge(validacion, on="device_id", how="left").fillna(tres_dias)
    target.set_index("device_id", inplace=True)
    target.to_pickle("../targets/targets_ventana_{}_auctions.pkl".format(validacion_nro))